In [191]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import re

# download list of stopwords from nltk lib.
stop_words = set(stopwords.words('english'))

import warnings
warnings.filterwarnings('ignore')


from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import pairwise_distances

from scipy.sparse import hstack

[nltk_data] Downloading package stopwords to /home/karol/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [189]:
def convert_to_string(row, column):
    if isinstance(row[column], list):
        # If the value is a list, join the strings using a comma
        subject_key_str = ' '.join(row[column])
    else:
        # If the value is a string, just return the string itself
        subject_key_str = row[column]
    return subject_key_str


def nlp_preprocessing(total_text, index, column, dataframe):
    if type(total_text) is str:
        string = ""
        for words in total_text.split():
            # remove the special chars like '"#$@!%^&*()_+-~?>< etc.
            word = ("".join(e for e in words if e.isalnum()))
            # Convert all letters to lower-case
            word = word.lower()
            # stop-word removal
            if not word in stop_words:
                string += word + " "
        dataframe[column][index] = string
    else:
        dataframe[column][index] = ""
        
        

def contains(sublist, item):
    for l in sublist:
        if re.search(r".*{}.*".format(item.lower()), l.lower()):
            return True
    return False
        

In [242]:
#read full book list and process subject_key text column

df = pd.read_json('data/downloaded.json')


# Apply convert_to_string function to each row of the 'subject_key' column
df['subject_key'] = df.apply(convert_to_string, args=('subject_key',), axis=1)
df['place_key'] = df.apply(convert_to_string, args=('place_key',), axis=1)


for index, row in df.iterrows():
    nlp_preprocessing(row['subject_key'], index, 'subject_key',df)
for index, row in df.iterrows():
    nlp_preprocessing(row['place_key'], index, 'place_key',df)

df=df.reset_index(drop=True)

df.iloc[[208]]

,title,author_name,first_publish_year,last_publish_year,place,place_key,subject,subject_key,person,person_key,language,isbn,key,url
208,Persians,Lloyd Llewellyn-Jones,2022,2022,"[Babylon, Egypt, Pasargadae, Persepolis]",babylon egypt pasargadae persepolis,"[Persia, ancient history, Asian / Middle Easte...",ancienthistory asianmiddleeasternhistory persia,"[Alexander, Artaxerxes, Cambyses, Cyrus, Dariu...","[alexander, artaxerxes, cambyses, cyrus, dariu...",[eng],9781541604230,/works/OL25338087W,https://openlibrary.org/works/OL25338087W


In [205]:
#df[df['place'].fillna('').apply(contains, item='babyl')]


In [67]:
#all_subjects=''
#for i in df['subject_key']:
#    all_subjects+=i

In [244]:
#read selected favorie book list and process subject_key text column

df_fav = pd.read_csv('data/my_favs.csv')
df_fav= df_fav.set_index('Unnamed: 0')
df_fav.index.name = None

#fav_subjects=''
#for i in df_fav['subject_key']:
#    fav_subjects+=i
 #print(fav_subjects)   
    
df_fav["subject_key"] = df_fav["subject_key"].str.replace(","," ")
df_fav["place_key"] = df_fav["place_key"].str.replace(","," ")


for index, row in df_fav.iterrows():
    nlp_preprocessing(row['subject_key'], index, 'subject_key',df_fav)
for index, row in df_fav.iterrows():
    nlp_preprocessing(row['place_key'], index, 'place_key',df_fav)
    
df_fav.head(1)
    




,title,author_name,first_publish_year,last_publish_year,place,place_key,subject,subject_key,person,person_key,language,isbn,key,url,favorite
208,Persians,Lloyd Llewellyn-Jones,2022,2022,"Babylon,Egypt,Pasargadae,Persepolis",babylon egypt pasargadae persepolis,"Persia,ancient history,Asian / Middle Eastern ...",ancienthistory asianmiddleeasternhistory persia,"Alexander,Artaxerxes,Cambyses,Cyrus,Darius,Her...","alexander,artaxerxes,cambyses,cyrus,darius,her...",eng,9781541604230,/works/OL25338087W,https://openlibrary.org/works/OL25338087W,True


In [245]:
df[df.index.isin(df_fav.index)]

,title,author_name,first_publish_year,last_publish_year,place,place_key,subject,subject_key,person,person_key,language,isbn,key,url
208,Persians,Lloyd Llewellyn-Jones,2022,2022,"[Babylon, Egypt, Pasargadae, Persepolis]",babylon egypt pasargadae persepolis,"[Persia, ancient history, Asian / Middle Easte...",ancienthistory asianmiddleeasternhistory persia,"[Alexander, Artaxerxes, Cambyses, Cyrus, Dariu...","[alexander, artaxerxes, cambyses, cyrus, dariu...",[eng],9781541604230,/works/OL25338087W,https://openlibrary.org/works/OL25338087W
866,Everyday Life in Ancient Mesopotamia,Jean Bottéro,2001,2001,[Iraq],iraq,"[Antiquities, Civilization, History, Ancient N...",ancientneareast antiquities bcetoc500ce civili...,None,None,[eng],9780801868627,/works/OL8396695W,https://openlibrary.org/works/OL8396695W
1560,Mesopotamia,Gwendolyn Leick,2001,2002,[Iraq],iraq,"[Civilization, Ancient Cities and towns, Iraq,...",ancientcitiesandtowns asiahistory civilization...,None,None,"[eng, spa]",9780140265743,/works/OL3961855W,https://openlibrary.org/works/OL3961855W
1673,Excavating an empire,Touraj Daryaee,2014,2014,[Iran],iran,"[History, Ancient History]",ancienthistory history,[Achaemenid dynasty (559-330 B.C)],[achaemenid_dynasty_(559-330_b.c)],[eng],9781568592985,/works/OL23318927W,https://openlibrary.org/works/OL23318927W
1689,The Persian empire,"Allen, Lindsay.",2005,2005,[Iran],iran,"[Civilization, History, History, ancient, Perz...",civilization histoire history historyancient p...,None,None,[eng],9780226014470,/works/OL5844839W,https://openlibrary.org/works/OL5844839W
1967,Persian art,Vladimir Loukonine,1998,2012,[Iran],iran,"[Ancient Art, Iranian Art, History, Catalogs, ...",ancientart art catalogs history iranianart,None,None,[eng],9781859951675,/works/OL20325135W,https://openlibrary.org/works/OL20325135W
2086,Arsacids and Sasanians,M. Rahim Shayegan,2011,2011,"[Rome, Iran]",iran rome,"[History, Sassanids, HISTORY / Ancient / Gener...",history historyancientgeneral iranhistoryto640...,[Arsacid dynasty (247 B.C.-224 A.D)],[arsacid_dynasty_(247_b.c.-224_a.d)],[eng],9780521766418,/works/OL16414417W,https://openlibrary.org/works/OL16414417W


# Create Bag of Words

In [183]:
subject_key_vectorizer = CountVectorizer()
subject_key_features   = subject_key_vectorizer.fit_transform(df['subject_key'])
#joblib.dump(subject_key_vectorizer, 'subject_key_vect.pkl')
subject_key_features.get_shape() 

(4901, 9873)

In [184]:
place_key_vectorizer = CountVectorizer()
place_key_features   = place_key_vectorizer.fit_transform(df['place_key'])
#joblib.dump(subject_key_vectorizer, 'subject_key_vect.pkl')
place_key_features.get_shape() 

(4901, 518)

In [222]:
all_features_df = hstack((subject_key_features, place_key_features)).tocsr()

## For more simultaneous input-books, create a new row in df that contains all the key words of all df_fav books together. Then create a new bag of words

In [254]:
df[df.index.isin(df_fav.index)].index

Index([208, 866, 1560, 1673, 1689, 1967, 2086], dtype='int64')

In [262]:
def bag_of_words_model(doc_id, num_results):
    # doc_id: apparel's id in given corpus
    
    # pairwise_dist will store the distance from given input apparel to all remaining apparels
    # the metric we used here is cosine, the coside distance is mesured as K(X, Y) = <X, Y> / (||X||*||Y||)
    # http://scikit-learn.org/stable/modules/metrics.html#cosine-similarity
    pairwise_dist = pairwise_distances(all_features_df,all_features_df[doc_id])
    
    # np.argsort will return indices of the smallest distances
    indices = np.argsort(pairwise_dist.flatten())[0:num_results]
    #pdists will store the smallest distances
    pdists  = np.sort(pairwise_dist.flatten())[0:num_results]

    #data frame indices of the 9 smallest distace's
    df_indices = list(df.index[indices])
    
    for i in range(0,len(indices)):
        # we will pass 1. doc_id, 2. title1, 3. title2, url, model
        #get_result(indices[i],data['title'].loc[df_indices[0]], data['title'].loc[df_indices[i]], data['medium_image_url'].loc[df_indices[i]], 'bag_of_words')
        print('TITLE :',df['title'].loc[df_indices[i]])
        print ('AUTHOR:', df['author_name'].loc[df_indices[i]])
        print ('Euclidean similarity with the query image :', pdists[i])


        print('='*60)

#call the bag-of-words model for a product to get similar products.
bag_of_words_model(1560, 10) # change the index if you want to.

TITLE : Mesopotamia
AUTHOR: Gwendolyn Leick
Euclidean similarity with the query image : 0.0
TITLE : Cornerstone
AUTHOR: Pedro Azara
Euclidean similarity with the query image : 2.0
TITLE : Fragments of Colossae
AUTHOR: Alan Cadwallader
Euclidean similarity with the query image : 2.449489742783178
TITLE : Ancient Town-Planning
AUTHOR: Francis John Haverfield
Euclidean similarity with the query image : 2.449489742783178
TITLE : From Mesopotamia to Iraq
AUTHOR: Hans Jörg Nissen
Euclidean similarity with the query image : 2.449489742783178
TITLE : Iraq
AUTHOR: Michael Wood
Euclidean similarity with the query image : 2.449489742783178
TITLE : Old Kingdom Town of Buhen
AUTHOR: David O'Connor
Euclidean similarity with the query image : 2.449489742783178
TITLE : Lost mythological cities of India
AUTHOR: Urmila Varma
Euclidean similarity with the query image : 2.6457513110645907
TITLE : Lost cities
AUTHOR: Maria Teresa Guaitoli
Euclidean similarity with the query image : 2.6457513110645907
TITL

In [188]:
df.columns

Index(['title', 'author_name', 'first_publish_year', 'last_publish_year',
       'place', 'place_key', 'subject', 'subject_key', 'person', 'person_key',
       'language', 'isbn', 'key', 'url'],
      dtype='object')

In [267]:
pd.set_option('display.max_colwidth', None)

In [269]:
df_fav

,title,author_name,first_publish_year,last_publish_year,place,place_key,subject,subject_key,person,person_key,language,isbn,key,url,favorite
208,Persians,Lloyd Llewellyn-Jones,2022,2022,"Babylon,Egypt,Pasargadae,Persepolis",babylon egypt pasargadae persepolis,"Persia,ancient history,Asian / Middle Eastern history",ancienthistory asianmiddleeasternhistory persia,"Alexander,Artaxerxes,Cambyses,Cyrus,Darius,Herodotus,Xerxes","alexander,artaxerxes,cambyses,cyrus,darius,herodotus,xerxes",eng,9781541604230,/works/OL25338087W,https://openlibrary.org/works/OL25338087W,True
866,Everyday Life in Ancient Mesopotamia,Jean Bottéro,2001,2001,Iraq,iraq,"Antiquities,Civilization,History,Ancient Near East,Social history,Mesopotamia,BCE to c 500 CE,History: World,Social life and customs,Iraq, history, to 634",ancientneareast antiquities bcetoc500ce civilization history historyworld iraqhistoryto634 mesopotamia socialhistory sociallifeandcustoms,NaN,NaN,eng,9780801868627,/works/OL8396695W,https://openlibrary.org/works/OL8396695W,True
1560,Mesopotamia,Gwendolyn Leick,2001,2002,Iraq,iraq,"Civilization,Ancient Cities and towns,Iraq, history, to 634,Asia, history,Iraq, antiquities,History",ancientcitiesandtowns asiahistory civilization history iraqantiquities iraqhistoryto634,NaN,NaN,"eng,spa",9780140265743,/works/OL3961855W,https://openlibrary.org/works/OL3961855W,True
1673,Excavating an empire,Touraj Daryaee,2014,2014,Iran,iran,"History,Ancient History",ancienthistory history,Achaemenid dynasty (559-330 B.C),achaemenid_dynasty_(559-330_b.c),eng,9781568592985,/works/OL23318927W,https://openlibrary.org/works/OL23318927W,True
1689,The Persian empire,"Allen, Lindsay.",2005,2005,Iran,iran,"Civilization,History,History, ancient,Perzische rijk,Histoire",civilization histoire history historyancient perzischerijk,NaN,NaN,eng,9780226014470,/works/OL5844839W,https://openlibrary.org/works/OL5844839W,True
1967,Persian art,Vladimir Loukonine,1998,2012,Iran,iran,"Ancient Art,Iranian Art,History,Catalogs,Art",ancientart art catalogs history iranianart,NaN,NaN,eng,9781859951675,/works/OL20325135W,https://openlibrary.org/works/OL20325135W,True
2086,Arsacids and Sasanians,M. Rahim Shayegan,2011,2011,"Rome,Iran",iran rome,"History,Sassanids,HISTORY / Ancient / General,Political science,Iran, history, to 640",history historyancientgeneral iranhistoryto640 politicalscience sassanids,Arsacid dynasty (247 B.C.-224 A.D),arsacid_dynasty_(247_b.c.-224_a.d),eng,9780521766418,/works/OL16414417W,https://openlibrary.org/works/OL16414417W,True
